# Obliczanie metryk generacji tekstu

In [1]:
predictions = [
    "Partycjonowanie zakresowe — podział tabeli na partycje według przedziałów wartości klucza (np. daty). Korzyści: - szybsze zapytania filtrujące po kluczu dzięki ograniczeniu skanowanych partycji (partition pruning) - łatwe operacje utrzymaniowe/retencja (drop/archiwizacja starych partycji) - mniejsze indeksy i szybsze czynności typu VACUUM/backup per partycja Kiedy stosować: - bardzo duże tabele, często filtrowane po zakresie (time-series, logi) - potrzeba szybkiego usuwania/archiwizacji starych danych",
    "eee nie wiem"
]
references = [
    "Partycjonowanie zakresowe dzieli dane na partycje według zakresów wartości klucza (np. daty). Ma sens gdy domena klucza jest duża - poprawia wydajność przez zmniejszenie konkurencji, umożliwia operacje na części danych (archiwizacja, backup, odtwarzanie) i redukuje rozmiar przetwarzanych obiektów.",
    "Semafor to zmienna nieujemna sterująca synchronizacją, z operacjami P (proberen) i V (verhogen). P zmniejsza semafor i może zablokować proces jeśli wartość ≤0, V zwiększa semafor i potencjalnie budzi oczekujące procesy."
]

predictions_multi = [
    "aaaa",
    "bbb",
    "ccc"
]
references_multi = [
    ["DDD", "ddd"],
    ["bbb"],
    ["ccc", "CCC"]
]

## BERTScore

In [2]:
from bert_score import score

P, R, F1 = score(cands=predictions, refs=references, lang="pl", verbose=True)
P, R, F1

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.19 seconds, 10.69 sentences/sec


(tensor([0.6764, 0.5871]), tensor([0.7344, 0.5263]), tensor([0.7042, 0.5551]))

In [3]:
score(cands=predictions_multi, refs=references_multi, lang="pl", verbose=True)

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.04 seconds, 130.14 sentences/sec


(tensor([0.7645, 1.0000, 1.0000]),
 tensor([0.7645, 1.0000, 1.0000]),
 tensor([0.7645, 1.0000, 1.0000]))

## ROUGE

In [4]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
scores

[{'rouge1': Score(precision=0.2898550724637681, recall=0.45454545454545453, fmeasure=0.35398230088495575),
  'rouge2': Score(precision=0.1323529411764706, recall=0.20930232558139536, fmeasure=0.16216216216216217),
  'rougeL': Score(precision=0.21739130434782608, recall=0.3409090909090909, fmeasure=0.2654867256637168)},
 {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0),
  'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
  'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}]

Nie ma wbudowanej obsługi wielu referencji

In [8]:
scores = [scorer.score(refs[0], pred) for refs, pred in zip(references_multi, predictions_multi)]
scores

[{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0),
  'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
  'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0),
  'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
  'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0),
  'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
  'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}]

## BLEU

In [5]:
from sacrebleu import corpus_bleu
bleu = corpus_bleu([predictions[0]], [[references[0]]])
bleu.score

11.62337454358603

In [16]:
corpus_bleu(predictions_multi, references_multi).score

0.0